In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os


In [ ]:
sys.path.insert(0, os.path.abspath("../src/"))
os.environ["CUDA_VISIBLE_DEVICES"]=""


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torchray.attribution.extremal_perturbation import extremal_perturbation, contrastive_reward, Perturbation
from torchray.benchmark import get_example_data, plot_example
from torchray.utils import get_device
import torch
from torchray.attribution.guided_backprop import guided_backprop

from pathlib import Path
from evaluation.eval_utils.compute_masks import compute_and_save_masks, vedaldi2019, compute_mask
from evaluation.eval_utils.compute_scores import get_model_and_data
from utils.helper import get_target_dictionary


In [ ]:
# Create data
nx = 100
nc = 10
c = 0
mask1 = np.random.rand(nx,nx)
seg1 = (np.random.rand(nx,nx)>0.5)*1

mask2 = np.ones([nx,nx])
mask3 = np.zeros([nx,nx])
mask4 = seg1.copy()

pvec1 = np.random.rand(nc)
pvec1 = pvec1 / np.sum(pvec1)

pvec2 = np.zeros(nc)
pvec2[c] = 1

pvec3 = np.ones(nc)/nc


In [ ]:
from evaluation.eval_utils.assessment_metrics import prob_entropy, prob_entropy, saliency, continuous_IOU, discrete_IOU, prob_sparsity

In [ ]:
prob_sparsity(pvec1), prob_sparsity(pvec2), prob_sparsity(pvec3)

In [ ]:
prob_entropy(pvec1), prob_entropy(pvec2), prob_entropy(pvec3)

In [ ]:
saliency(pvec1, c, mask1), saliency(pvec2, c, mask1), saliency(pvec3, c, mask1)

In [ ]:
continuous_IOU(mask1, seg1), continuous_IOU(mask4, seg1), continuous_IOU(mask2, seg1), continuous_IOU(mask3, seg1)

In [ ]:
discrete_IOU(mask1, seg1), discrete_IOU(mask4, seg1), discrete_IOU(mask2, seg1), discrete_IOU(mask3, seg1), 


# Metrics using torchray

In [ ]:
from torchray.attribution.grad_cam import grad_cam
from torchray.benchmark import get_example_data, plot_example
from torchray.attribution.rise import rise



In [ ]:
path_masks = Path("../evaluation/")
data_path = Path("../datasets/VOC2007/")
dataset_name = "VOC"
model_name = "vgg16"
model_path = "../src/checkpoints/pretrained_classifiers/vgg16_voc.ckpt"

if dataset_name=="COCO":
    path_segmentation = path_masks / Path('coco_segmentations')
elif dataset_name=="VOC":
    path_segmentation = Path('../datasets/VOC2007/VOCdevkit/VOC2007/SegmentationClass/')

    

In [ ]:
model, data_module = get_model_and_data(data_path, dataset_name, model_name, model_path)

In [ ]:
device = get_device()
model.to(device);


In [ ]:
method = "guided_backprop"

path_outputs = path_masks / Path('{}_{}_{}/'.format(dataset_name, model_name, method))

count = compute_and_save_masks(model, data_module, path_segmentation, path_outputs, method=method)
print(count)

In [ ]:
it = data_module.test_dataloader()

In [ ]:

# # Obtain example data.
# model, x, category_id, _ = get_example_data()
c = 36
for s in it:
    img, meta = s
    x = img
    if dataset_name == "COCO":
        category_id = meta[0]["targets"]
    elif dataset_name == "VOC":
        target_dict = get_target_dictionary(include_background_class=False)
        objects = meta[0]['annotation']['object']
        category_id = [target_dict[e["name"]] for e in objects]
    c -= 1
    if c<0:
        break
x = x.to(device)

In [ ]:
# Grad-CAM backprop.
if model_name == "vgg16":
    feature_extractor = model.feature_extractor[-1]
elif model_name == "resnet50":
    feature_extractor = model.feature_extractor[-2][-1]
    
saliency = grad_cam(model, x, 1, saliency_layer = feature_extractor, resize=True)

# Plots.
plot_example(x, saliency, 'grad-cam backprop', category_id)
np.max(saliency.detach().cpu().numpy()), np.min(saliency.detach().cpu().numpy())